# Kernel-Image-Primal-Dual: A Practical Guide

This notebook provides the code for the blog post *"Kernel-Image-Primal-Dual: Navigating SDP Formulations"*. We use `cvxpy` to implement the four different semidefinite programming (SDP) formulations for checking if a polynomial is a sum of squares (SOS).

We will focus on the example from the blog post: a univariate polynomial of degree 4:
$$ p(x) = p_0 + p_1x + p_2x^2 + p_3x^3 + p_4x^4 $$

This polynomial is a sum of squares if $p(x) = \mathbf{v}(x)^\top \mathbf{X} \mathbf{v}(x)$ for some positive semidefinite Gram matrix $\mathbf{X} \succeq 0$, where the monomial basis is $\mathbf{v}(x) = [1, x, x^2]^\top$.

In [ ]:
import cvxpy as cp
import numpy as np
import time

# Set numpy print options for better readability
np.set_printoptions(precision=4, suppress=True)

## Setup: Defining the Problem Matrices

First, we define the matrices $\mathbf{A}_i$, $\mathbf{B}_j$, and $\mathbf{Y}(\mathbf{p})$ that constitute the building blocks of our four formulations. These are derived directly from the coefficient-matching conditions described in the blog post.

In [ ]:
# The coefficient-matching matrices A_i for the constraint <X, A_i> = p_i
A = [
    np.array([[1, 0, 0], [0, 0, 0], [0, 0, 0]]), # A_0 for p_0
    np.array([[0, 1, 0], [1, 0, 0], [0, 0, 0]]), # A_1 for p_1
    np.array([[0, 0, 1], [0, 1, 0], [1, 0, 0]]), # A_2 for p_2
    np.array([[0, 0, 0], [0, 0, 1], [0, 1, 0]]), # A_3 for p_3
    np.array([[0, 0, 0], [0, 0, 0], [0, 0, 1]])  # A_4 for p_4
]

# The matrix B_1 for the null space of the coefficient-matching operator
B = [
    np.array([[0, 0, -0.5], [0, 1, 0], [-0.5, 0, 0]])
]

# A function to construct the matrix Y(p) from the polynomial coefficients p
def get_Y(p_coeffs):
    p0, p1, p2, p3, p4 = p_coeffs
    return np.array([
        [p0,    p1/2, 0],
        [p1/2,  p2,   p3/2],
        [0,     p3/2, p4]
    ])


## The Four Formulations

In [ ]:
def solve_primal_kernel(p_coeffs, A_matrices):
    """Solves the (P-K) Primal Kernel Problem."""
    X = cp.Variable((3, 3), symmetric=True)
    
    constraints = [
        cp.trace(A_matrices[i] @ X) == p_coeffs[i] for i in range(len(p_coeffs))
    ]
    constraints.append(X >> 0)
    
    problem = cp.Problem(cp.Minimize(0), constraints)
    start_time = time.time()
    problem.solve(solver=cp.CVXOPT)
    end_time = time.time()

    dual_variables = [c.dual_value for c in constraints]
    
    return X.value, dual_variables, problem.status, end_time - start_time

def solve_primal_image(p_coeffs, B_matrices):
    """Solves the (P-I) Primal Image Problem."""
    s = cp.Variable(len(B_matrices))
    Y_p = get_Y(p_coeffs)
    
    # In this example, there is only one slack variable s_0
    constraint = (Y_p + s[0] * B_matrices[0] >> 0)
    
    problem = cp.Problem(cp.Minimize(0), [constraint])
    start_time = time.time()
    problem.solve(solver=cp.CVXOPT)
    end_time = time.time()
    
    # Reconstruct the Gram matrix X if a solution is found
    X_sol = None
    if s.value is not None:
        X_sol = Y_p + s.value[0] * B_matrices[0]
        
    return X_sol, constraint.dual_value, problem.status, end_time - start_time

def solve_dual_kernel(p_coeffs, A_matrices):
    """Solves the (D-K) Dual Kernel Problem."""
    lambda_vars = cp.Variable(len(p_coeffs))
    
    constraints = [
        lambda_vars @ p_coeffs == -1,
        cp.sum([lambda_vars[i] * A_matrices[i] for i in range(len(p_coeffs))]) >> 0
    ]
    
    problem = cp.Problem(cp.Minimize(0), constraints)
    start_time = time.time()
    problem.solve(solver=cp.CVXOPT)
    end_time = time.time()

    dual_variables = [c.dual_value for c in constraints]
    return lambda_vars.value, dual_variables, problem.status, end_time - start_time

def solve_dual_image(p_coeffs, B_matrices):
    """Solves the (D-I) Dual Image Problem."""
    X = cp.Variable((3, 3), symmetric=True)
    Y_p = get_Y(p_coeffs)
    
    constraints = [
        cp.trace(Y_p.T @ X) == -1,
        cp.trace(B_matrices[0].T @ X) == 0,
        X >> 0
    ]
    
    problem = cp.Problem(cp.Minimize(0), constraints)
    start_time = time.time()
    problem.solve(solver=cp.CVXOPT)
    end_time = time.time()

    dual_variables = [c.dual_value for c in constraints]
    return X.value, dual_variables, problem.status, end_time - start_time

## Case 1: A Feasible SOS Problem

Let's test the formulations on a polynomial that is known to be a sum of squares, for example:
$p(x) = (1+x^2)^2 = 1 + 2x^2 + x^4$.

The coefficient vector is $\mathbf{p} = [1, 0, 2, 0, 1]$.

In [ ]:
p_feasible = np.array([1., 0., 2., 0., 1.])

print("--- Testing Feasible SOS Problem ---")
print(f"p(x) = {p_feasible[0]} + {p_feasible[2]}x^2 + {p_feasible[4]}x^4\n")

# (P-K)
X_pk, d_pk, status, t = solve_primal_kernel(p_feasible, A)
print(f"(P-K) Primal Kernel: Status='{status}', Time={t:.4f}s")
print(f"Gram matrix X:\n{X_pk}")
print(f"Dual values:\n{d_pk}\n")

# (P-I)
X_pi, d_pi, status, t = solve_primal_image(p_feasible, B)
print(f"(P-I) Primal Image:  Status='{status}', Time={t:.4f}s")
print(f"Gram matrix X:\n{X_pi}")
print(f"Dual values:\n{d_pi}\n")

# (D-K)
# For a feasible primal, the dual is also feasible. A non-zero solution is a certificate.
# However, CVXOPT may find the trivial solution lambda=0 if the primal is strictly feasible.
L_dk, d_dk, status, t = solve_dual_kernel(p_feasible, A)
print(f"(D-K) Dual Kernel:   Status='{status}', Time={t:.4f}s")
print(f"Lambda variables:\n{L_dk}")
print(f"Dual values:\n{d_dk}\n")

# (D-I)
# For a feasible primal, the dual is also feasible. X=0 is a trivial solution.
X_di, d_di, status, t = solve_dual_image(p_feasible, B)
print(f"(D-I) Dual Image:    Status='{status}', Time={t:.4f}s")
print(f"Dual matrix X:\n{X_di}")
print(f"Dual values:\n{d_di}\n")


As expected, both primal forms find a valid Gram matrix $\mathbf{X} \succeq 0$. Note that they might find different valid matrices, as the solution is not unique. Both dual problems are infeasible. 

## Case 2: An Infeasible (Not SOS) Problem

Now, let's try a polynomial that is not a sum of squares because it is not non-negative. For a univariate polynomial, not being non-negative is a sufficient condition for not being SOS.

$p(x) = 2x^4 - 3x^2 - 1$.

The coefficient vector is $\mathbf{p} = [-1, 0, -3, 0, 2]$. This problem is infeasible because $p_0 = -1$, which requires the top-left element of the Gram matrix ($X_{00}$) to be negative, violating the positive semidefinite condition.

In [ ]:
p_infeasible = np.array([-1., 0., -3., 0., 2.])

print("--- Testing Infeasible (Not SOS) Problem ---")
print(f"p(x) = {p_infeasible[0]} + {p_infeasible[2]}x^2 + {p_infeasible[4]}x^4\n")

# (P-K)
X_pk, d_pk, status, t = solve_primal_kernel(p_infeasible, A)
print(f"(P-K) Primal Kernel: Status='{status}', Time={t:.4f}s")
print(f"Gram matrix X: {X_pk}\n")

# (P-I)
X_pi, d_pi, status, t = solve_primal_image(p_infeasible, B)
print(f"(P-I) Primal Image:  Status='{status}', Time={t:.4f}s")
print(f"Gram matrix X: {X_pi}\n")

# (D-K)
# For an infeasible primal, the dual problem provides a certificate of infeasibility.
L_dk, d_dk, status, t = solve_dual_kernel(p_infeasible, A)
print(f"(D-K) Dual Kernel:   Status='{status}', Time={t:.4f}s")
print(f"Lambda variables:\n{L_dk}\n")

# (D-I)
# When (P-I) is infeasible, its dual is typically unbounded. CVXPY reports this as 'infeasible'.
# This is because the feasibility problem we wrote has an optimal value of -inf.
X_di, d_di, status, t = solve_dual_image(p_infeasible, B)
print(f"(D-I) Dual Image:    Status='{status}', Time={t:.4f}s")
print(f"Certificate matrix X: {X_di}\n")

Here, the results are flipped. The primal problems (P-K) and (P-I) are correctly identified as infeasible, meaning no SOS decomposition exists. 

Conversely, the dual problems (D-K) and (D-I) are feasible. These solutions act as a formal *certificates* proving that the primal problems are infeasible. 